In [35]:
import math 

def parse_integer(int_packet):
    reached_end = False
    remaining_packet = int_packet
    binary_total = []

    length_read = 0

    while not reached_end:
        next_5 = remaining_packet[:5]
        length_read += 5
        remaining_packet = remaining_packet[5:]

        binary_total.append(next_5[1:])

        if next_5[0] == "0":
            reached_end = True

    return int("".join(binary_total), 2), length_read

def parse_packet(packet) -> tuple[int, int, int]:

    total_sum = 0
    version_sum = 0

    version = int(packet[:3], 2)
    type_id = int(packet[3:6], 2)

    version_sum += version
    length_read = 6


    # integer literal
    if type_id == 4:
        integer, l_read = parse_integer(packet[6:])
        length_read += l_read
        total_sum += integer
    
    # operator
    else:
        length_type_id = int(packet[6], 2)
        length_read += 1
        result_array_from_children = []

        # 15 bits
        if length_type_id == 0:
            number_of_bits_in_sub_packets = int(packet[7:7+15], 2)
            length_read += 15
            cursor = 7 + 15

            while cursor < number_of_bits_in_sub_packets + 7 + 15:
                v_sum, l_read, total_sum = parse_packet(packet[cursor:])
                version_sum += v_sum
                length_read += l_read
                cursor += l_read
                result_array_from_children.append(total_sum)

        # 11 bits
        else:
            number_of_sub_packets = int(packet[7:7+11], 2)
            length_read += 11
            cursor = 7 + 11

            for n in range(number_of_sub_packets):
                v_sum, l_read, total_sum = parse_packet(packet[cursor:])
                version_sum += v_sum
                cursor += l_read
                length_read += l_read
                result_array_from_children.append(total_sum)

        # operator
        if type_id == 0:
            total_sum = sum(result_array_from_children)
        elif type_id == 1:
            total_sum = math.prod(result_array_from_children)
        elif type_id == 2:
            total_sum = min(result_array_from_children)
        elif type_id == 3:
            total_sum = max(result_array_from_children)
        elif type_id == 5:
            total_sum = 1 if result_array_from_children[0] > result_array_from_children[1] else 0
        elif type_id == 6:
            total_sum = 1 if result_array_from_children[0] < result_array_from_children[1] else 0
        elif type_id == 7:
            total_sum = 1 if result_array_from_children[0] == result_array_from_children[1] else 0
            

    return version_sum, length_read, total_sum

def main(file, type):
    rows = file.read().splitlines()

    for row in rows:
        binary_arr = []
        for char in row:
            binary_arr.append("{0:08b}".format(int(char, 16))[4:])
        binary = "".join(binary_arr)

        v_sum_1, _, total = parse_packet(binary)

        print(f"{type} {row} \nPart 1: {v_sum_1}\nPart 2: {total}\n")

with open("test.txt") as input_file:
    main(input_file, "Test")

with open("input.txt") as input_file:
    main(input_file, "Real")

Test 9C005AC2F8F0 
Part 1: 16
Part 2: 0

Real 020D78804D397973DB5B934D9280CC9F43080286957D9F60923592619D3230047C0109763976295356007365B37539ADE687F333EA8469200B666F5DC84E80232FC2C91B8490041332EB4006C4759775933530052C0119FAA7CB6ED57B9BBFBDC153004B0024299B490E537AFE3DA069EC507800370980F96F924A4F1E0495F691259198031C95AEF587B85B254F49C27AA2640082490F4B0F9802B2CFDA0094D5FB5D626E32B16D300565398DC6AFF600A080371BA12C1900042A37C398490F67BDDB131802928F5A009080351DA1FC441006A3C46C82020084FC1BE07CEA298029A008CCF08E5ED4689FD73BAA4510C009981C20056E2E4FAACA36000A10600D45A8750CC8010989716A299002171E634439200B47001009C749C7591BD7D0431002A4A73029866200F1277D7D8570043123A976AD72FFBD9CC80501A00AE677F5A43D8DB54D5FDECB7C8DEB0C77F8683005FC0109FCE7C89252E72693370545007A29C5B832E017CFF3E6B262126E7298FA1CC4A072E0054F5FBECC06671FE7D2C802359B56A0040245924585400F40313580B9B10031C00A500354009100300081D50028C00C1002C005BA300204008200FB50033F70028001FE60053A7E93957E1D09940209B7195A56BCC75AE7F18D46E273882402CCD006A600